In [1]:
import pandas as pd
import tqdm
import openai
import nltk
import os

from pyprojroot import here
from moral_foundations_llms import utils
from textwrap import dedent

In [2]:
comments = pd.read_csv(here('data/aita_comments_Aug17_labels_update_Aug31.csv'))
filled = pd.read_csv(here('data/aita_nan_comments_labels_gpt.csv'))

In [3]:
comments = comments.drop(columns=['flair_text_y'])
comments = comments.rename(
    columns={'flair_text_x': 'flair_text',
             'score_x': 'score_comment',
             'score_y': 'score_submission'})
comments = comments[comments['comment_author'] != 'AutoModerator'] 
comments = comments[comments['comment_author'] != 'AmItheAsshole-ModTeam']

In [4]:
comments = comments.merge(right=filled[['comment_permalink', 'gpt_comment_label']],
               how='left',
               on='comment_permalink')

In [5]:
comments['comment_label'].replace(
    {'NTA (Not the Asshole)': 'NTA',
     'NTA or "Not the Asshole"': 'NTA',
     "ESH (Everyone Sucks Here)": "ESH",
     
     "YWBTA": "YTA",
     "NTAH": "NTA",
     "YTA (You're the Asshole)": "YTA",
     "Label: YTA (You're the Asshole)": "YTA",
     "YTA/ESH": "YTA",
     "NTA/NAH": "NTA",
     "NAH/INFO": "NAH",
     "Label: YTA": "YTA",
     "ESH, but even slight YTA.": "ESH",
     "The comment expresses multiple judgements, including NTA for expecting the friend not to make jokes about the miscarriage, and YTA for expecting the friend to want to be around the baby": "YTA",
     'Based on the comment you provided, it seems to be expressing the judgement of YTA or "You\'re the Asshole". The commenter states that the OP is at fault for not recognizing that the child\'s behavior was normal and for overreacting by involving the MIL.': "YTA",
     "I apologize for the confusion. I am a tool designed to evaluate social media comments and assign them a label based on whether the commenter is seen as the asshole (YTA), not the asshole (NTA), everyone sucks (ESH), no assholes (NAH), not enough information (INFO), or if no specific judgement is being rendered (NONE). I will analyze the comment you provide and assign the appropriate label based on the content of the comment.         1\nNTA to NAH": "NTA",
    }, inplace=True)

In [6]:
comments['comment_label'] = comments['comment_label'].fillna(comments['gpt_comment_label'])
comments.loc[~comments['comment_label'].str.contains('YTA|NTA|INFO|NONE|ESH|INFO|NAH', regex=True), 'comment_label'] = 'NONE'
comments.loc[comments['comment_label'].str.contains('NONE'), 'comment_label'] = 'NONE'
comments.loc[comments['comment_label'].str.contains('INFO'), 'comment_label'] = 'INFO'
comments.loc[comments['comment_label'].str.contains('YTA'), 'comment_label'] = 'YTA'
comments.loc[comments['comment_label'].str.contains('NTA'), 'comment_label'] = 'NTA'
comments.loc[comments['comment_label'].str.contains('ESH'), 'comment_label'] = 'ESH'

In [7]:
comments['comment_label'].value_counts(dropna=False)

NTA     306405
YTA      99228
NONE     82065
ESH      13241
INFO     13138
NAH       9683
Name: comment_label, dtype: int64

In [10]:
comments.to_csv(here('data/aita_all_comments_labeled.csv'), index=False)

/opt/homebrew/Caskroom/miniforge/base/envs/nlp/lib/python3.9/site-packages/pyprojroot/pyprojroot.py:51: UserWarning: Path doesn't exist: /Users/psachdeva/projects/moral_foundations_llms/data/aita_all_comments_labeled.csv
  warnings.warn("Path doesn't exist: {}".format(path))
